In [1]:
import sqlite3
import pandas as pd
from fpdf import FPDF

In [2]:
# creating a class for accessing data from fictional.db
class DBtables:
    def __init__(self, dbpath):
        self.dbpath = dbpath
        self.con = sqlite3.connect(dbpath)
        self.cur = self.con.cursor()

    def __del__(self):
        self.con.close()
    
    # method for viewing/exporting Clients table data
    def read_clients(self):
        answer = input('Enter 1 to export all to Excel or any other key to show on screen (only last 10 clients will be shown): ')
        if answer == '1':
            orders = pd.read_sql_query('SELECT * FROM Clients;', self.con)
            orders.to_excel('all_clients.xlsx')
            return('Client data exported to Excel.')
        else:
            return pd.read_sql_query('SELECT * FROM Clients;', self.con).tail(10)
    
    # a group of db table column search methods. I tried several approaches to search whole table
    # in one database query but I am not there yet
    def clients_by_name(self, keyword):
        self.cur.execute("SELECT * FROM Clients WHERE Name LIKE (?);", ('%'+keyword+'%',))
        a = self.cur.fetchall()
        return a

    def clients_by_details(self, keyword):
        self.cur.execute("SELECT * FROM Clients WHERE Details LIKE (?);", ('%'+keyword+'%',))
        a = self.cur.fetchall()
        return a
        
    def clients_by_address(self, keyword):
        self.cur.execute("SELECT * FROM Clients WHERE Address LIKE (?);", ('%'+keyword+'%',))
        return self.cur.fetchall()

    def clients_by_phone(self, keyword):
        self.cur.execute("SELECT * FROM Clients WHERE Phone LIKE (?);", ('%'+keyword+'%',))
        a = self.cur.fetchall()
        return a

    def clients_by_notes(self, keyword):
        self.cur.execute("SELECT * FROM clients WHERE Notes LIKE (?);", ('%'+keyword+'%',))
        a = self.cur.fetchall()
        return a

    # method for viewing Orders table data
    def read_orders_all(self):
        answer = input('Enter 1 to export all to Excel or any other key to show on screen (only last 5 orders will be shown): ')
        if answer == '1':
            orders = pd.read_sql_query('SELECT * FROM Orders;', self.con)
            orders.to_excel('all_orders.xlsx')
            return('Orders exported to Excel.')
        else:
            return pd.read_sql_query('SELECT * FROM Orders;', self.con).tail(5)
    
    # method for showing orders by selected (one) activity type
    def orders_by_type(self):
        answer = input('Choose activity type (V, DA or M): ').lower()
        self.cur.execute("SELECT * FROM orders WHERE TypeId = (?);", (answer.upper(),))
        a = self.cur.fetchall()
        return a

    # method for getting activity type according to order's unique DateID
    def get_type(self, dateid):
        self.cur.execute("SELECT TypeId FROM orders WHERE DateId = (?);", (dateid,))
        a = self.cur.fetchall()
        return a   

    # method for viewing Order status by client name (assuming unique for now)
    def order_status(self):
        name = input("Enter client's name to view order status for this client: ")
        self.cur.execute("""SELECT Name, Date, Description, "Price €", OrderSt
                            FROM Orders o 
                            JOIN Clients c
                            ON c.ClientID = o.ClientID 
                            WHERE c.Name LIKE (?);""", ('%'+ name + '%',))
        a = self.cur.fetchall()
        return a[0]
    
    # method for getting Order status details according to ClientId - to be used in invoice generation
    def order_status_id(self, clid):
        self.cur.execute("""SELECT Name, Date, Description, "Price €", OrderSt
                            FROM Orders o 
                            JOIN Clients c
                            ON c.ClientID = o.ClientID 
                            WHERE c.ClientId = (?);""", (clid,))
        a = self.cur.fetchall()
        return a
    
    # method for getting client details according to order unique DateId- to be used in invoice generation
    def order_client(self, dateid):
        self.cur.execute("""SELECT Name, Details
                            FROM Clients c 
                            JOIN Orders o
                            ON c.ClientID = o.ClientID 
                            WHERE o.DateId = (?);""", (dateid,))
        a = self.cur.fetchall()
        return a

    # method for counting yearly TypeId instances - to be used in invoice generation
    def type_count(self, iseries):
        self.cur.execute('SELECT COUNT(InvNo) FROM Orders WHERE InvNo LIKE (?);', (iseries+'%',))
        a = self.cur.fetchall()
        return a

    # method for updating client notes only (assuming unique client name for now)
    def update_client_notes(self):
        name = input("Enter client's name for whom notes to be changed: ")
        notes = input("Enter new notes (to change other information, use delete and insert functions): ")
        self.cur.execute('UPDATE clients SET notes = (?) WHERE name LIKE (?);', (notes, '%' + name + '%'))
        self.con.commit()
        return("Clients notes updated.")

    # method for updating some order details
    def update_order_details(self):
        order = input("Enter order dateID - only description, price, order status and units can be changed: ")
        descr = input("Enter new description: ")
        price = input("Enter new price: ")
        orderst = input("Enter new order status (available: none, conf, pre, full, paid): ")
        un = input("Enter new work units (working hours, quantity): ")
        if len(order) == 8 and price.isnumeric() == True:
            try:
                self.cur.execute("""UPDATE orders SET Description = (?),
                                    "Price €" = (?), OrderSt = (?), Unit = (?) 
                                    WHERE dateid = (?);""", (descr, price, orderst, order, un))
                self.con.commit()
                return("Order details updated.")
            except ValueError:
                return("No details were updated.")
        else:
            return("No details were updated, please enter correct details.")

    # method for adding a client record
    def add_client(self):
        numberof = pd.read_sql_query("""SELECT ClientID FROM Clients
                                         ORDER BY ClientID DESC LIMIT 1;""", self.con).iat[0,0]
        clid = input(f"Enter client's id (last one in the database is {numberof}): ")
        name = input("Enter client's name: ")
        details = input("Enter client's details: ")
        address = input("Enter client's address, email: ")
        phone = input("Enter client's phone number: ")
        notes = input("Enter further notes, comments: ")
        self.cur.execute("""INSERT into Clients
                            (ClientID, Name, Details, Address, Phone, Notes)
                            VALUES ((?), (?), (?), (?), (?), (?));""", (clid, name, details, address, phone, notes))
        answer = input(f'Submitting a new row <{clid}, {name}, {details}, {address}, {phone}, {notes}> to database, press "y" to confirm: ').lower()
        if answer == 'y':
            self.con.commit()
            return("New client details added to the database.")
        else:
            return("Details NOT added to the database.")
    
    # methods for deleting records
    # getting client's name by provided client Id
    def client_name_id(self, clid):
        (answer,) = self.cur.execute("SELECT Name FROM Clients WHERE ClientID = (?);", (clid,))
        return answer[0]

    # method to delete client's record unless there are orders associated with them
    def delete_client(self):
        clid = input("""It is advised to delete clients only when updating their information by adding a new line. 
                            For other purposes client notes can be updated. If you wish to continue, enter ClientID: """)
        if DBtables.order_status_id(self, int(clid)) != []:
            print('Cannot delete - this client has order records in Orders table. If the records are not needed, you can delete them first.')
        else:
            name = DBtables.client_name_id(self, clid)
            if input(f"Enter 'y' if you want to delete client's {name} details: ").lower() == 'y':
                self.cur.execute('DELETE FROM Clients WHERE ClientID = (?);', (clid,))
                self.con.commit()
                return("One row deleted as requested.")
                
    # method to delete order record by database unique order ID (not DateId)
    def delete_order(self):
        ordid = input("""Beware of deleting order information you may need for accounting purposes. 
                        If you still wish to proceed, enter Order #: """)
        self.cur.execute('DELETE FROM Orders WHERE "#" = (?);', (ordid,))
        if ordid.isnumeric() == True:
            return("Order deleted as requested.")
        else:
            return("Order has not been deleted.")

    # method to get single order record by DateId - currently for generating invoice only
    def inv_line(self):
        order = input('Enter number (DateID) of order to be invoiced: ')
        return pd.read_sql_query(f'SELECT * FROM Orders WHERE DateID LIKE "{order}"', self.con) 

    # method to insert a new invoice number (overwrites with existing if it exists)
    def insert_inv_no(self, new_no, dateid):
        self.cur.execute('UPDATE Orders SET InvNo = (?) WHERE DateId = (?);', (new_no, dateid))
        self.con.commit()
        return
    

In [3]:
# the multiple db query search Clients table function... 
def search_clients(db):
    keyword = input("Enter client's name or another search word: ")
    lines = []
    x = 0
    if DBtables.clients_by_name(db, keyword) != [] and DBtables.clients_by_name(db, keyword) not in lines:
        lines.append(DBtables.clients_by_name(db, keyword))
    else:
        x +=1
    if DBtables.clients_by_details(db, keyword) != [] and DBtables.clients_by_details(db, keyword) not in lines:
        lines.append(DBtables.clients_by_details(db, keyword))
    else:
        x +=1
    if DBtables.clients_by_address(db, keyword) != [] and DBtables.clients_by_address(db, keyword) not in lines:
        lines.append(DBtables.clients_by_address(db, keyword))
    else:
        x +=1
    if DBtables.clients_by_phone(db, keyword) != [] and DBtables.clients_by_phone(db, keyword) not in lines:
        lines.append(DBtables.clients_by_phone(db, keyword))
    else:
        x +=1
    if DBtables.clients_by_notes(db, keyword) != [] and DBtables.clients_by_notes(db, keyword) not in lines:
        lines.append(DBtables.clients_by_notes(db, keyword))
    else:
        x +=1
    if x < 5:
        a = []
        for i in range(len(lines[0])):
            a.append(lines[0][i])
        return a
    else:
        return('Keyword not found.')

In [4]:
# function definitions to get user input for invoice generation
# getting invoice type for invoice
def inv_title():
    choice = int(input('Enter 1 for Pro-forma invoice, 2 for Invoice: '))
    match choice:
        case 1:
            return 'PRO-FORMA INVOICE'
        case 2:
            return 'INVOICE'

# getting invoice date from a user
def in_date():
    return input('Enter invoice issue date YYYY-MM-DD: ')

# getting invoice amount in words from a user
# I saw there are functions for int to string conversion but not there yet
def amount_words(query):
    amount = query.iloc[0]['Price €']
    return input(f'Enter amount {amount} in words...: ')

# getting confirmation from the user whether the invoice has been paid
def inv_paid():
    answer = input("Enter 'y' if invoice has been paid: ")
    if answer.lower() == 'y':
        return 2
    else:
        return 1   

# getting invoice paid date from a user
def paiddate():
    return input('Enter invoice paid date YYYY-MM-DD: ')

# function to gather user entry variables for invoice generation
def inv_variables_user():
    db = DBtables("fictional.db")
    query = db.inv_line()
    dateid = query.iloc[0]['DateID']
    inv_no = query.iloc[0]['InvNo']  
    title = inv_title()
    inv_date = in_date()
    amount = amount_words(query)
    if title == 2:
        paid = inv_paid()
        datepaid = paiddate()
    else:
        paid = 1
        datepaid = ''
    return query, dateid, inv_no, title, inv_date, amount, paid, datepaid

# getting invoice series for invoice number (consists of a series number and year YY number)
def inv_series(dateid):
    db = DBtables("fictional.db")
    part1 = db.get_type(dateid)[0][0] 
    part2 = dateid[:2]
    return part1 + part2   

# inserts new invoice number using DBtables insert_inv_no function and returns it (or the existing one)
def inv_number(str1, dateid, inv_no):
    if inv_no == None or inv_no == '':
        db = DBtables("fictional.db")
        series_count = int(db.type_count(str1)[0][0])
        if series_count < 10:
            new_no = str1 + '0' + str(series_count + 1)
        else:
            new_no = str1 + str(series_count + 1)
        db.insert_inv_no(new_no, dateid)
        return str(series_count + 1)
    else:
        return inv_no[(len(inv_no)-2):]

# getting client information for invoice
def buyer_details(dateid):
    db = DBtables("fictional.db")
    details = db.order_client(dateid)
    buyer, buyer_info = details[0]
    return buyer, buyer_info

# getting calculated or database variables for invoice
def inv_variables_calc(query, dateid, inv_no):
    str1 = inv_series(dateid)
    str2 = inv_number(str1, dateid, inv_no)
    inv_date = query.iloc[0]['Date'].replace('.', '-')
    buyer = buyer_details(dateid)[0]
    buyer_info = buyer_details(dateid)[1]
    descript = query.iloc[0]['Description']
    unit = query.iloc[0]['Unit']
    price = query.iloc[0]['Price €']
    return str1, str2, inv_date, buyer, buyer_info, descript, unit, price

In [5]:
def get_invoice():
    #  currently default variables
    seller = 'Lala Lalala'
    seller_info = 'Reg No 6543424'
    seller_acc = '1234 1234 1234 1234'
    items = 1
    
    #calling functions to gather invoice information
    query, dateid, inv_no, title, inv_date, amount, paid, datepaid = inv_variables_user()
    str1, str2, inv_date, buyer, buyer_info, descript, unit, price = inv_variables_calc(query, dateid, inv_no)
    sum = price # see TODO list re multiple orders on one invoice

    # creating invoice template
    invoice1 = FPDF('P', 'mm', 'A4')
    invoice1.set_margins(15,15,15)
    invoice1.add_page()
    invoice1.set_font("Times", 'B', 12)
    invoice1.set_draw_color(0,0,0)
    invoice1.image('fictlogo.png', 15, 15)
    invoice1.cell(0, 10, f'{title}', ln = 1, align = 'C')
    invoice1.cell(0, 10, f"Series: {str1} No: {str2}", ln = 2, align = 'C')
    invoice1.cell(0, 30, f'{inv_date}', ln = 2, align = 'C')
    invoice1.cell(0, 10, 'Seller', ln = 0, align = 'L')
    invoice1.set_x(130)
    invoice1.cell(10, 10, 'Buyer', ln = 1, align = 'L')
    invoice1.set_font("Times", '', 12)
    invoice1.set_xy(15,75)
    invoice1.multi_cell(60, 7, f'{seller}\n{seller_info}\nBank Acc. {seller_acc}', align = 'L')
    invoice1.set_xy(130,75)
    invoice1.multi_cell(60, 7, f'{buyer}\n{buyer_info}', align = 'L')
    invoice1.set_xy(15,110)
    invoice1.set_font("Times", 'B', 12)
    invoice1.cell(18, 10, 'Number', border = 1, ln = 0, align = 'C')
    invoice1.cell(50, 10, 'Description', border = 1, ln = 0, align = 'C')
    invoice1.cell(20, 10, 'Unit', border = 1, ln = 0, align = 'C')
    invoice1.cell(22, 10, 'Quantity', border = 1, ln = 0, align = 'C')
    invoice1.cell(34, 10, 'Unit Price, Eur', border = 1, ln = 0, align = 'C')
    invoice1.cell(36, 10, 'Total for Item, Eur', border = 1, ln = 0, align = 'C')
    invoice1.set_font("Times", '', 12)
    invoice1.set_xy(15,120)
    invoice1.cell(18, 20, '1', border = 1, ln = 0, align = 'C')
    invoice1.cell(50, 20, f'{descript}', border = 1, align = 'C')
    invoice1.cell(20, 20, f'{unit}', border = 1, ln = 0, align = 'C')
    invoice1.cell(22, 20, f'{items}', border = 1, ln = 0, align = 'C')
    invoice1.cell(34, 20, f'{price}', border = 1, ln = 0, align = 'C')
    invoice1.cell(36, 20, f'{price}', border = 1, ln = 0, align = 'C') 
    invoice1.set_xy(15, 140)
    invoice1.set_font("Times", 'B', 12)    
    invoice1.cell(144, 10, 'Total invoice amount', border = 1, ln = 0, align = 'R')
    invoice1.cell(36, 10, f'{sum}', border = 1, ln = 0, align = 'C')
    invoice1.set_font("Times", '', 12)
    invoice1.set_xy(18,160)
    invoice1.multi_cell(50, 7, f'Invoice amount in words:\n{amount}', border = 0, align = 'L')
    match paid:
        case 1:
            pass
        case 2:
            invoice1.set_xy(18, 180)
            invoice1.cell(50, 10, f'PAID: {datepaid}', ln = 1, align = 'L')
    invoice1.set_xy(18, 200)
    invoice1.cell(70, 10, f'Invoice issued by {seller}.', ln = 1, align = 'L')
    invoice1.output(f"Invoice_{dateid}_{inv_date}.pdf")
    
    answer = input('Invoice has been generated. Press "y", if you want to generate another invoice: ')
    if answer.lower() == 'y':
        get_invoice()
    else:
        pass

In [6]:
# menu function definitions
def main_menu():
    a = int(input("""Welcome to order processing application.\n
                    Please enter:\n
                    1 to view client and order information,\n
                    2 to change client or order details,\n
                    3 to create invoices for the orders: """))
    return a

def view_menu():
    a = int(input("""Please enter:\n
                    1 to view client information or export to Excel,\n
                    2 to search client records,\n
                    3 to view order details or export to Excel,\n
                    4 to view order list by specific activity,\n
                    5 to view order status,\n
                    6 to return to the main menu: """))
    return a

def change_menu():
    a = int(input("""Please enter:\n
                    1 to change client notes,\n
                    2 to change order details,\n
                    3 to add new client data,\n
                    4 to remove client details (when changing notes not enough),\n
                    5 to delete an order,\n
                    6 to return to the main menu: """))
    return a

In [7]:
# menu function "switchboard"
def main_exec():
    match main_menu():
        case 1:
            view_exec()
        case 2:
            change_exec()
        case 3:
            get_invoice()       

def view_exec():
    match view_menu():
        case 1:
            db = DBtables("fictional.db")
            print(db.read_clients())
        case 2:
            db = DBtables("fictional.db")
            query = search_clients(db)
            for i in range(len(query)):
                print(query[i])
        case 3:
            db = DBtables("fictional.db")
            query= db.read_orders_all()
            # for i in range(len(query)):
            print(query)
        case 4:
            db = DBtables("fictional.db")
            query = db.orders_by_type()
            for i in range(len(query)):
                print(query[i])
        case 5:
            db = DBtables("fictional.db")
            print(db.order_status())
        case 6:
            main_exec()

def change_exec():
    match change_menu():
        case 1:
            db = DBtables("fictional.db")
            print(db.update_client_notes())
        case 2:
            db = DBtables("fictional.db")
            print(db.update_order_details())
        case 3:
            db = DBtables("fictional.db")
            print(db.add_client())
        case 4:
            db = DBtables("fictional.db")
            print(db.delete_client())
        case 5:
            db = DBtables("fictional.db")
            print(db.delete_client())
        case 6:
            main_exec()

In [10]:
# Run the program...
main_exec()

Cannot delete - this client has order records in Orders table. If the records are not needed, you can delete them first.
None


In [9]:
# TODO add or improve error handling
# TODO add a method to add a new order to the database
# TODO add a method to generate DateID and Agreement number when adding a new order
# TODO adjust methods/functions to account for multiple same client name in the database
# TODO improve the exit/return to main menu options
# TODO add an option to include several orders for the same client within one invoice
# TODO add an option to have a longer order description in the invoice
# TODO add a function to convert numbers to words so the user doesn't have to enter
# TODO try alternative functions with less database connections
# TODO add a function to plot monthly income per activity etc
# TODO improve function for search in whole Clients table. if success, add a similar function for Orders table